# Worksheet 05

Name: Jonathan Thomas
UID: U81823882

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

- Points nearest to 0: [0, 0.5]
- Points nearest to 2: [1.5, 2, 6, 6.5, 7]

New centroids: [0.25, 4.6]

- Points nearest to 0.25: [0, 0.5, 1.5, 2]
- Points nearest to 4.6: [6, 6.5, 7]

New centroids: [1, 6.5]

- Points nearest to 1: [0, 0.5, 1.5, 2]
- Points nearest to 6.5: [6, 6.5, 7]

Final clusters: [0, 0.5, 1.5, 2], [6, 6.5, 7]



b) Describe in plain english what the cost function for k means is.

It is the sum of all distances from each cluster where those distances are the euclidean distances between each point and the centroid.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

If the points are completely uniform in shape it can come down to the initial selection of points.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes. We must converge as we are continuously decreasing the cost of the function whilst never allowing it to increase.

e) Follow along in class the implementation of Kmeans

In [2]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))
        
    def is_unassigned(self, i):
        return self.assignment[i] == -1
    
    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]
        
    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), size=self.k, replace=False)]
    
    def are_centers_diff(self, c1, c2):
        for i in range(len(c1)):
            if c1[i] not in c2:
                return True
        return False
    
    def assign(self, centers):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_assign = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if temp_dist > new_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist
    
    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j,axis=0))
        
        return np.array(centers)

    def dist(self, x, y):
        return sum((x - y) ** 2) ** (1/2)

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.snap(centers)
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
            print (new_centers)
        return
            
kmeans = KMeans(X, 4)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

[[ 0.93703017 -4.3139755 ]
 [ 1.4688642   1.2262519 ]
 [ 2.69408178 -3.81299013]
 [-2.5131847   1.49370647]]
[[ 0.9255334  -4.24622594]
 [ 1.36194348  1.30382385]
 [ 2.63171879 -3.63223886]
 [-2.66744643  1.52630187]]
[[ 0.99102631 -4.30543204]
 [ 1.29796725  1.26432606]
 [ 2.64783164 -3.53024329]
 [-2.75338814  1.58980866]]
[[ 1.05447434 -4.40854007]
 [ 1.26512502  1.24441713]
 [ 2.61630642 -3.38268935]
 [-2.79587234  1.62518468]]
[[ 1.11385013 -4.52269782]
 [ 1.26512502  1.24441713]
 [ 2.54373601 -3.24316321]
 [-2.79587234  1.62518468]]
[[ 1.12066428 -4.56511935]
 [ 1.26610724  1.26378621]
 [ 2.4614405  -3.17598087]
 [-2.79587234  1.62518468]]
[[ 1.12066428 -4.56511935]
 [ 1.26137141  1.28143239]
 [ 2.44621274 -3.11960387]
 [-2.79587234  1.62518468]]
[[ 1.12066428 -4.56511935]
 [ 1.25234224  1.29773417]
 [ 2.44503358 -3.0609286 ]
 [-2.79587234  1.62518468]]
[[ 1.12066428 -4.56511935]
 [ 1.25234224  1.29773417]
 [ 2.44503358 -3.0609286 ]
 [-2.79587234  1.62518468]]
